![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [25]:
import pandas as pd
import numpy as np

# Start coding here.*..

In [26]:
df = pd.read_csv("bank_marketing.csv")
print(df.info())
object_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in object_cols:
    print(col)
    print("--------------")
    print(df[col].value_counts())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

,client_id,age,day,contact_duration,number_contacts,previous_campaign_contacts,cons_price_idx,euribor_three_months
count,41188.00000,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,20593.50000,40.02406,15.730334,258.285010,2.567593,0.172963,93.575664,3.621291
std,11890.09578,10.42125,8.889867,259.279249,2.770014,0.494901,0.578840,1.734447
min,0.00000,17.00000,1.000000,0.000000,1.000000,0.000000,92.201000,0.634000
25%,10296.75000,32.00000,8.000000,102.000000,1.000000,0.000000,93.075000,1.344000
50%,20593.50000,38.00000,16.000000,180.000000,2.000000,0.000000,93.749000,4.857000
75%,30890.25000,47.00000,23.000000,319.000000,3.000000,0.000000,93.994000,4.961000
max,41187.00000,98.00000,31.000000,4918.000000,56.000000,7.000000,94.767000,5.045000


In [27]:
df.drop_duplicates()
# modified: job, education
df['job'] = df['job'].str.replace('.', '_')
df['education'] = df['education'].str.replace('.', '_')
df['education'][df['education']=='unknown'] = np.NaN

In [28]:
# Convert: credit_default, mortgage, previous_outcome, campaign_outcome
df['credit_default'] = df['credit_default'] == 'yes'
df['mortgage'] = df['mortgage'] == 'yes'
df['previous_outcome'] = df['previous_outcome'] == 'success'
df['campaign_outcome'] =  df['campaign_outcome'] == 'yes'


In [29]:
# Convert month names to corresponding intergers
month = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
df['month'] = df['month'].map(month)
# add the column 'year'
df['year'] = 2012

In [30]:
df['last_contact_date'] = pd.to_datetime(df[["year", "month", "day"]])

In [31]:
# Create dataframes
client_cols = ['client_id', 
              'age',
              'job',
              'marital',
              'education',
              'credit_default',
              'mortgage'
             ]
campaign_cols = ['client_id',
                 'number_contacts',
                'contact_duration',
                'previous_campaign_contacts',
                'previous_outcome',
                'campaign_outcome',
                'last_contact_date']
economics_cols = ['client_id',
                 'cons_price_idx',
                 'euribor_three_months'
                 ]

client = df[client_cols]
campaign = df[campaign_cols]
economics = df[economics_cols]

In [32]:
# check client
print(client.info())
print(client.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None
   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y           False     False
1          1   57   services  married  high_school           False     False
2          2   37   services  married  high_school           False      True
3          3   40     admin_  married     basic_6y           False     False
4          4   56   services  married  high_sch

In [33]:
# check campaign
print(campaign.info())
print(campaign.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB
None
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2012-05-13
1          1                1  ...             False         2012-05-19
2          2                1

In [34]:
# check economics
print(economics.info())
print(economics.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None
   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


In [35]:
# to csv
client.to_csv("client.csv",index=False)
campaign.to_csv("campaign.csv",index=False)
economics.to_csv("economics.csv",index=False)